In [ ]:
! pip install pytorch-lightning --quiet

     |████████████████████████████████| 563kB 8.6MB/s 
     |████████████████████████████████| 829kB 12.6MB/s 
     |████████████████████████████████| 92kB 11.3MB/s 
     |████████████████████████████████| 276kB 27.3MB/s 


In [ ]:
import os
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.metrics import Accuracy 
from pytorch_lightning.metrics.functional.classification import to_categorical 
from torch.utils.data import random_split
from sklearn.metrics import classification_report , confusion_matrix

import torchtext
from torchtext import  data , vocab
from torchtext.datasets import SST


In [ ]:
class GloveEmbedding(pl.LightningModule):
    def __init__(self,
                vocab_size, 
                embedding_dim , 
                embedding_weight , 
                pad_index):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim,padding_idx=pad_index)
        self.embedding.weight = nn.Parameter(embedding_weight, requires_grad=False)
     
    def forward(self,x):
        x = self.embedding(x)
        return x

In [ ]:
class LSTMEncoder(pl.LightningModule):
    def __init__(self,
                embedding_dim=300, 
                hidden_dim=168, 
                num_classes=5, 
                num_layers=1 , 
                dropout=0.5,
                embedding=None,
                learning_rate=1e-3,
                weight_decay=1e-3):
        super().__init__()

        self.embedding = embedding        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim , num_layers=num_layers , dropout=dropout)
        self.lin = nn.Linear(hidden_dim, num_classes)
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay

        self.true_labels = []
        self.pred_labels = []
     
    def forward(self,x):
        x , length = x
        x = self.embedding(x)
        x = nn.utils.rnn.pack_padded_sequence(x,length.cpu())
        packed_output, (hidden,cell) = self.lstm(x)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        x = hidden[-1,:,:]
        x = F.elu(x, alpha=0.1)
        x = self.lin(x)

        return x
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate , weight_decay=self.weight_decay)
        return optimizer
    
    def training_step(self,batch,batch_idx):
        x, y = batch.text, batch.label-1

        logits = self(x)
        loss = F.cross_entropy(logits,y)
        accuracy = Accuracy()
        acc = accuracy(torch.tensor(logits).cpu(),torch.tensor(y).cpu())

        pbar = {'training_acc': acc}

        return {'loss' : loss , 'progress_bar':pbar}
    
    def training_epoch_end(self , train_step_outputs):
        avg_loss = torch.stack([x['loss'] for x in train_step_outputs]).mean()
        avg_acc =  torch.stack([x['progress_bar']['training_acc'] for x in train_step_outputs]).mean()

        tqdm_dict = {'train_loss': avg_loss , 'train_acc': avg_acc}
        return {
                'progress_bar': tqdm_dict,
                'log': {'train_loss': avg_loss , 'train_acc': avg_acc},
                }
    
    def validation_step(self , batch , batch_idx):
        result = self.training_step(batch,batch_idx)
        result['progress_bar']['val_acc'] = result['progress_bar']['training_acc']
        return result
    
    def validation_epoch_end(self , val_step_outputs):
        avg_loss = torch.stack([x['loss'] for x in val_step_outputs]).mean()
        avg_acc =  torch.stack([x['progress_bar']['val_acc'] for x in val_step_outputs]).mean()

        tqdm_dict = {'val_loss': avg_loss , 'val_acc': avg_acc}
        return {
                'progress_bar': tqdm_dict,
                'log': {'val_loss': avg_loss , 'val_acc': avg_acc},
                }
    
    def test_step(self , batch , batch_idx):
        x, y = batch.text, batch.label-1
        logits = self(x)
        loss = F.cross_entropy(logits,y)
        logits = torch.tensor(logits)
        accuracy = Accuracy()
        acc = accuracy(logits.cpu(), torch.tensor(y).cpu())

        pbar = {'test_acc': acc }
        return {'loss' : loss , 'progress_bar':pbar}

    def test_epoch_end(self , test_step_outputs):
        avg_loss = torch.stack([x['loss'] for x in test_step_outputs]).mean()
        avg_acc =  torch.stack([x['progress_bar']['test_acc'] for x in test_step_outputs]).mean()

        tqdm_dict = {'test_loss': avg_loss , 'test_acc': avg_acc}
        return {
                'progress_bar': tqdm_dict,
                'log': {'test_loss': avg_loss , 'test_acc': avg_acc},
                }
    

In [ ]:
  class SSTDataModule(pl.LightningDataModule):
    def __init__(self , batch_size=64 , fine_grained = True , gpu=0):
        super().__init__()
        self.batch_size = batch_size
        self.fine_grained = fine_grained
        self.vectors = None
        self.TEXT = data.Field(lower=True, include_lengths=True)
        self.LABEL = data.Field(sequential=False)
        self.gpu = gpu
       
        self.num_classes = 3 

        if self.fine_grained : 
            self.num_classes = 5
        
        print(f"-- num classes = {self.num_classes}")
        self.train,self.val, self.test = SST.splits(self.TEXT, self.LABEL , fine_grained=self.fine_grained)
        
    def train_dataloader(self):
        return self.train_iter
    
    def val_dataloader(self):
        return self.val_iter
    
    def test_dataloader(self):
        return self.test_iter

def setup_Glove(self,stage=None):
    if stage is None :
        self.LABEL.build_vocab(self.train)
        self.TEXT.build_vocab(self.train, vectors=vocab.GloVe(), unk_init = torch.Tensor.normal_)
        self.vectors = self.TEXT.vocab.vectors
        print(f"Label : {self.LABEL.vocab.stoi}")
        device = 'cpu'
        if self.gpu > 0 :
            device = 'cuda' if torch.cuda.is_available() else 'cpu'
        
        self.train_iter, self.val_iter , self.test_iter = data.BucketIterator.splits(
            (self.train,self.val, self.test), 
            batch_size=self.batch_size,
            sort_key=lambda x: len(x.text), # for tabular data  
            sort_within_batch = True,
            device=device
        )

SSTDataModule.setup = setup_Glove
dm = SSTDataModule(batch_size=32 , gpu=1)
dm.setup() 

-- num classes = 5
downloading trainDevTestTrees_PTB.zip


trainDevTestTrees_PTB.zip: 100%|██████████| 790k/790k [00:00<00:00, 2.78MB/s]


extracting


.vector_cache/glove.840B.300d.zip: 2.18GB [16:53, 2.15MB/s]                            
100%|█████████▉| 2195593/2196017 [03:50<00:00, 10529.67it/s]

Label : defaultdict(<function _default_unk_index at 0x7f9d8f2e0d08>, {'<unk>': 0, 'positive': 1, 'negative': 2, 'neutral': 3, 'very positive': 4, 'very negative': 5})


In [ ]:
pl.seed_everything(1234)

hparam = {
        "num_classes" : dm.num_classes ,
        "hidden_dim" : 168 ,
        "num_layers" : 1,
        "learning_rate" : 1e-3,
        "weight_decay" : 1e-3,
    }

embedding = GloveEmbedding(vocab_size=len(dm.TEXT.vocab),embedding_dim=300,embedding_weight= dm.vectors,pad_index=dm.TEXT.vocab.stoi[dm.TEXT.pad_token])
model = LSTMEncoder(embedding_dim=300,
                            hidden_dim=hparam["hidden_dim"],
                            num_classes=hparam["num_classes"],
                            num_layers=hparam["num_layers"],
                            embedding=embedding,
                            learning_rate=hparam["learning_rate"],
                            weight_decay=hparam["weight_decay"])

# training
trainer = pl.Trainer(gpus=1, max_epochs=20 , progress_bar_refresh_rate=50)
trainer.fit(model, dm)
trainer.test(model)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:61: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type           | Params
---------------------------------------------
0 | embedding | GloveEmbedding | 4 M   
1 | lstm      | LSTM           | 315 K 
2 | lin       | Linear         | 845   


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The validation_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {progress_bar:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)
/usr/local/li

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:79: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:81: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The testing_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)


--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.4246), 'test_loss': tensor(1.4346, device='cuda:0')}
--------------------------------------------------------------------------------



[{'test_acc': 0.4245535731315613, 'test_loss': 1.434584140777588}]